In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping#to stop overfitting

from sklearn.metrics import classification_report, confusion_matrix

In [2]:
%%writefile requirements.txt
tensorflow
torch
opencv-python
scikit-learn
matplotlib
pandas
numpy

Writing requirements.txt


In [3]:
!pip install -r requirements.txt

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

Mounted at /content/drive
/content/drive/MyDrive


In [5]:
!unzip archive.zip  # or your ZIP name

Archive:  archive.zip
replace content/.config/.last_opt_in_prompt.yaml? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
import tensorflow as tf

DATA_DIR = "/content/drive/MyDrive/content/Diabetic_Balanced_Data"
IMG_SIZE = (64,64)
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR + "/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR + "/val",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

Found 34792 files belonging to 5 classes.
Found 9940 files belonging to 5 classes.


In [7]:
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    layers.Rescaling(1./255),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))
val_ds = val_ds.map(lambda x, y: (x/255.0, y))


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, Flatten
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(64,64,3)),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(64, (3,3), activation="relu"),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.25),

    Conv2D(128, (3,3), activation="relu"),
    BatchNormalization(),
    MaxPooling2D(),

    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(128, activation="relu"),
    Dense(5, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 62, 62, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 29, 29, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 12, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 127,813 (499.27 KB)

 Trainable params: 127,365 (497.52 KB)

 Non-trainable params: 448 (1.75 KB)

In [10]:
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=[early_stop]
)

Epoch 1/5
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 6755s 6s/step - accuracy: 0.3114 - loss: 1.4946 - val_accuracy: 0.2080 - val_loss: 1.7887
Epoch 2/5
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 581s 533ms/step - accuracy: 0.3684 - loss: 1.4059 - val_accuracy: 0.2210 - val_loss: 2.0942
Epoch 3/5
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 565s 519ms/step - accuracy: 0.3826 - loss: 1.3709 - val_accuracy: 0.2821 - val_loss: 1.6385
Epoch 4/5
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 560s 514ms/step - accuracy: 0.4020 - loss: 1.3421 - val_accuracy: 0.2897 - val_loss: 1.6989
Epoch 5/5
1088/1088 ━━━━━━━━━━━━━━━━━━━━ 561s 515ms/step - accuracy: 0.4188 - loss: 1.3164 - val_accuracy: 0.3513 - val_loss: 1.4518


In [11]:
#evalution matrix
from sklearn.metrics import classification_report
import numpy as np

y_true = np.concatenate([y.numpy() for x,y in val_ds], axis=0)
y_pred = model.predict(val_ds)

y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_true, axis=1)

print(classification_report(y_true, y_pred))


311/311 ━━━━━━━━━━━━━━━━━━━━ 67s 213ms/step
              precision    recall  f1-score   support

           0       0.22      0.17      0.19      2000
           1       0.50      0.00      0.01      1940
           2       0.20      0.62      0.30      2000
           3       0.11      0.00      0.01      2000
           4       0.21      0.21      0.21      2000

    accuracy                           0.20      9940
   macro avg       0.25      0.20      0.14      9940
weighted avg       0.25      0.20      0.14      9940



In [12]:
#GRAD-CAM (Explainability – REQUIRED)
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt

def grad_cam(model, img_array, layer_name="conv2d_2"):
    grad_model = tf.keras.models.Model(
        model.input,
        [model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_idx = tf.argmax(predictions[0])
        loss = predictions[:, class_idx]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))

    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(conv_outputs * pooled_grads, axis=-1)

    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    return heatmap